In [1]:
from pathlib import Path 


class ProjectPaths:
    """Centralized path management for the flood detection project"""
    
    def __init__(self, project_dir: Path):
        self.project_dir = project_dir
        # self.image_code = image_code
        
        
        # Main working directories
        self.dataset_dir = self.project_dir / "data" /  "4final" / "dataset"
        self.training_dir = self.project_dir / "data" / "4final" / "training"
        self.predictions_dir = self.project_dir / "data" / "4final" / 'predictions'
        
        # Data subdirectories
        self.images_dir = self.dataset_dir / 'S1Hand'
        self.labels_dir = self.dataset_dir / 'LabelHand'
        
        # CSV files
        self.train_csv = self.dataset_dir / "flood_train_data.csv"
        self.val_csv = self.dataset_dir / "flood_valid_data.csv"
        self.test_csv = self.dataset_dir / "flood_test_data.csv"
        
        # Checkpoint directories (consolidated)
        self.ckpt_input_dir = project_dir / "checkpoints" / 'ckpt_INPUT'
        self.ckpt_training_dir = self.project_dir / "checkpoints" / "ckpt_training"
        
        # Config files
        self.main_config = project_dir / "configs" / "floodaiv2_config.yaml"
        self.minmax_config = project_dir / "configs" / "global_minmax_INPUT" / "global_minmax.json"
        
        # Environment file
        self.env_file = self.project_dir / ".env"
    
    def get_inference_paths(self, sensor: str = 'sensor', date: str = 'date', tile_size: int =512, threshold: float = 0.5,  image_code: str = '0000', output_filename: str = '_name') -> dict:
        """Get inference-specific paths"""
        save_tiles_path = self.predictions_dir / f'{image_code}_tiles'
        return {
            'predict_input': self.project_dir / "data" / "4final" / "predict_input",
            'pred_tiles_dir': self.predictions_dir / f'{image_code}_predictions',
            'save_tiles_path': save_tiles_path,
            'extracted_dir': self.predictions_dir / f'{image_code}_extracted',
            'file_list': self.predictions_dir / "predict_tile_list.csv",
            'stitched_image': self.predictions_dir / f'{sensor}_{image_code}_{date}_{tile_size}_{threshold}_{output_filename}_WATER_AI.tif',
            'metadata_path': save_tiles_path / 'tile_metadata.json'
        }
    
    def get_training_paths(self):
        """Get training/testing specific paths"""
        return {
            'save_tiles_path': self.dataset_dir,
            'metadata_path': self.dataset_dir / 'tile_metadata_pth.json'
        }
    
    def validate_paths(self, job_type: str):
        """Validate that required paths exist for the given job type"""
        errors = []
        required_paths = []
        if job_type in ('train', 'test'):
            required_paths = [
                (self.dataset_dir, "Dataset directory"),
                (self.images_dir, "Images directory"),
                (self.labels_dir, "Labels directory"),
            ]
            
            if job_type == 'train':
                required_paths.extend([
                    (self.train_csv, "Training CSV"),
                    (self.val_csv, "Validation CSV")
                ])
            elif job_type == 'test':
                required_paths.append((self.test_csv, "Test CSV"))
                
        elif job_type == 'inference':
            # Inference paths are created dynamically, less validation needed
            pass
            
        # Always check checkpoint folder
        required_paths.append((self.ckpt_input_dir, "Checkpoint input directory"))
        
        for path, description in required_paths:
            if not path.exists():
                errors.append(f"{description} not found: {path}")
        
        # Check for checkpoint files
        if not any(self.ckpt_input_dir.rglob("*.ckpt")):
            errors.append(f"No checkpoint files found in: {self.ckpt_input_dir}")
            
        return errors

In [5]:
project_dir = Path('/Users/alexwebb/laptop_coding/floodai/InferSAR')
paths = ProjectPaths(project_dir=project_dir)

inf_pths = paths.get_inference_paths(sensor='S1A', date='20230101', tile_size=512, threshold=0.5, image_code='0001', output_filename='test_output')
print(inf_pths['predict_input'])

/Users/alexwebb/laptop_coding/floodai/InferSAR/data/4final/predict_input
